In [1]:
!python -m pip install --user --upgrade pip
!pip install --user --upgrade pandas matplotlib numpy scikit-learn tensorflow keras

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [3]:
#Check if the install was successful
!which dsl-compile

In [4]:
# Import Kubeflow SDK
import kfp
import kfp.dsl as dsl
import kfp.components as comp

In [5]:
import os


# where the outputs are stored
out_dir = str(os.getcwd())
model_file = 'modela'
holdout = True

In [6]:
os.getcwd()

'/home/jovyan/Volve_ML/new pipeline'

In [7]:
def train_tranform(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    # Download the dataset and split into training and test data. 
    data = pd.read_csv("https://raw.githubusercontent.com/Josepholaidepetro/Volve_ML/main/data/train.csv")
    
    # Preprocess
    data.drop(['DEPTH', 'BS', 'RD', 'ROP', 'RM', 'DRHO'], axis=1, inplace=True)
    
    # If Nan, drop
    data.dropna(inplace=True)
    
    # transform the RT to logarithmic
    data['RT'] = np.log10(data['RT'])
    
    # perform a yeo-johnson transform of the train dataset
    ptrain = PowerTransformer(method='yeo-johnson')
    train_df_yj = ptrain.fit_transform(data.drop('DT', axis=1))
    train_df_yj_norm = pd.DataFrame(train_df_yj, columns=data.columns.drop('DT'))
    y_train = data['DT']
    
    #Save the train_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/train_tranform_data', 'wb') as f:
        pickle.dump((train_df_yj_norm,  y_train), f)

In [8]:
train_tranform(out_dir)

In [9]:
def test_tranform(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    # Download the dataset and split into training and test data. 
    data = pd.read_csv("https://raw.githubusercontent.com/Josepholaidepetro/Volve_ML/main/data/test.csv")
    
    # Preprocess
    data.drop(['DEPTH', 'BS', 'ROP', 'DRHO'], axis=1, inplace=True)
    
    # If Nan, drop
    data.dropna(inplace=True)
    
    # transform the RT to logarithmic
    data['RT'] = np.log10(data['RT'])
    
    # perform a yeo-johnson transform of the train dataset
    ptest = PowerTransformer(method='yeo-johnson')
    test_df_yj = ptest.fit_transform(data.drop('DT', axis=1))
    test_df_yj_norm = pd.DataFrame(test_df_yj, columns=data.columns.drop('DT'))
    y_test = data['DT']
    
    #Save the test_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/test_data', 'wb') as f:
        pickle.dump((test_df_yj_norm,  y_test), f)

In [10]:
test_tranform(out_dir)

/home/jovyan/.local/lib/python3.6/site-packages/sklearn/preprocessing/_data.py:3234: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [11]:
import os
print(os.listdir())

['maven_analytics.ipynb', 'maven_new_kbflow.ipynb', 'Outlier_removal_train_valid', 'Outlier_removal_train_data', 'valid_tranform_train_data', 'holdout', 'dt_prediction_kubeflow.zip', '.ipynb_checkpoints', 'modela', 'test_data', 'valid_data', '...', 'train_tranform_data']


In [12]:
def Outlier_removal(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    
    from sklearn.svm import OneClassSVM
    import numpy as np
    import pandas as pd
    
    # Load and unpack the train_data
    with open(f'{data_path}/train_tranform_data','rb') as f:
        train_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    train_df_yj_norm,  y_train = train_data

    # Method 4: One-class SVM
    svm = OneClassSVM(nu=0.13)
    yhat = svm.fit_predict(train_df_yj_norm)
    mask = yhat != -1
    train_df_svm = train_df_yj_norm[mask]
    y_train_svm = y_train[mask]

        
    # prepare train data for modelling
    X_train = train_df_svm.copy().drop(['label'], axis=1)
    y_train = y_train_svm.copy()
    
    
    #Save the train_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/Outlier_removal_train_data', 'wb') as f:
        pickle.dump((X_train,  y_train), f)
        
    
    #Save the train_data to be used for splitting as a pickle file to be used by the predict component.
    with open(f'{data_path}/Outlier_removal_train_valid', 'wb') as f:
        pickle.dump((train_df_svm.copy(), y_train_svm.copy()), f)
    

In [13]:
Outlier_removal(out_dir)

In [14]:
def valid_tranform(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    # Load and unpack the full Outlier_removal_train_valid
    with open(f'{data_path}/Outlier_removal_train_valid','rb') as f:
        X_train,  y_train= pickle.load(f)
    train_split = pd.concat([X_train.reset_index(drop=True),  y_train.reset_index(drop=True)], axis=1)
    
    # Separate the data into X_train and y_train.
    X_train2 = train_split[train_split['label'] < 0].drop(['label', 'DT'], axis=1)
    y_train2 = train_split[train_split['label'] < 0]['DT']
    
    # Separate the data into X_valid and y_valid.
    X_valid = train_split[train_split['label'] > 0].drop(['label', 'DT'], axis=1)
    y_valid = train_split[train_split['label'] > 0]['DT']
    
    #Save the valid_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/valid_data', 'wb') as f:
        pickle.dump((X_valid,  y_valid), f)
        
    #Save the train_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/valid_tranform_train_data', 'wb') as f:
        pickle.dump((X_train2,  y_train2), f)

In [15]:
valid_tranform(out_dir)

In [16]:
def model_building(data_path, model_file, holdout):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    
    # Load and unpack the full Outlier_removal_train_data
    with open(f'{data_path}/Outlier_removal_train_data','rb') as f:
        Outlier_removal_train_data = pickle.load(f)
    # Separate the X_train from y_train.
    X_train,  y_train = Outlier_removal_train_data
    
    # Load and unpack the valid_tranform_train_data
    with open(f'{data_path}/valid_tranform_train_data','rb') as f:
        valid_tranform_train_data = pickle.load(f)
    # Separate the X_train from y_train.
    X_train_val,  y_train_val = valid_tranform_train_data
    
    # Load and unpack the valid_data
    with open(f'{data_path}/valid_data','rb') as f:
        valid_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_valid,  y_valid = valid_data
    
    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_test, y_test = test_data
    
    # Define the model using Keras
    tf.random.set_seed(1)
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units =128, activation='relu', input_dim=X_train.shape[1]))
    model.add(keras.layers.Dense(units =128, activation='relu'))
    model.add(keras.layers.Dense(units =256, activation='relu'))
    model.add(keras.layers.Dense(units =256, activation='relu'))
    model.add(keras.layers.Dense(units =1, activation='linear'))

    model.summary()

    model.compile(optimizer = 'adam', loss='mean_squared_error')

    
    if holdout is False:
        # Run a training job
        model.fit(X_train, y_train, batch_size=20 , epochs=5 )
    else:
        # Run a training job
        model.fit(X_train_val, y_train_val, batch_size=20 , epochs=20, validation_data=(X_valid, y_valid))
        
    model.save(f'{data_path}/{model_file}')

    
    #Save holdout
    with open(f'{data_path}/holdout', 'wb') as f:
        pickle.dump((holdout), f)

In [17]:
regressor = model_building(out_dir, model_file, holdout)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 116,609
Trainable params: 116,609
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
448/448 [==============================] - 3s 5ms/step - loss: 1104.4002 - val_loss: 99.7906
Epoch 2/20
4

In [18]:
def model_evaluate(data_path, model_file, holdout):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    
    # Load the saved model
    model = keras.models.load_model(f'{data_path}/{model_file}')

    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
    # Separate the X_test from y_test.
    X_test, y_test = test_data
    
    
    # Load and unpack the valid_data
    with open(f'{data_path}/valid_data','rb') as f:
        valid_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_valid,  y_valid = valid_data
    
    if holdout is False:
        #Evaluate the model and print the results
        score = model.evaluate(X_test, y_test, verbose=0)
        print("mse of Well 3: {}".format(score))
    else:
        score = model.evaluate(X_valid,y_valid, verbose=0)
        print("mse of Well 3: {}".format(score))


In [19]:
model_evaluate(out_dir, model_file, holdout)

mse of Well 3: 19.754016876220703


In [20]:
def predict(data_path, model_file):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    
    # Load the saved model
    regressor = keras.models.load_model(f'{data_path}/{model_file}')
    
    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
    # Separate the X_test from y_test.
    test_df_yj_norm,  y_test = test_data

    # make predictions.
    y_pred = regressor.predict(test_df_yj_norm)

    
    with open(f'{data_path}/result1', 'wb') as f:
        pickle.dump(y_pred, f)
        
    with open(f'{data_path}/result.txt', 'w') as result:
        result.write(" Prediction: {}, Actual: {} ".format(y_pred,y_test))
    
    print('Prediction has be saved successfully!')

In [21]:
# Create train and predict lightweight components.
train_tranform_op = comp.func_to_container_op(train_tranform , base_image = "tensorflow/tensorflow:latest-gpu-py3")
test_tranform_op = comp.func_to_container_op(test_tranform , base_image = "tensorflow/tensorflow:latest-gpu-py3")
Outlier_removal_op = comp.func_to_container_op(Outlier_removal , base_image = "tensorflow/tensorflow:latest-gpu-py3")
valid_tranform_op = comp.func_to_container_op(valid_tranform , base_image = "tensorflow/tensorflow:latest-gpu-py3")
train_op = comp.func_to_container_op(model_building , base_image = "tensorflow/tensorflow:latest-gpu-py3")
valid_op = comp.func_to_container_op(model_evaluate , base_image = "tensorflow/tensorflow:latest-gpu-py3")
predict_op = comp.func_to_container_op(predict , base_image = "tensorflow/tensorflow:latest-gpu-py3")

## Build Kubeflow Pipeline

In [22]:
#Create a client to enable communication with the Pipelines API server.
client = kfp.Client()

In [23]:
# Define the pipeline
@dsl.pipeline(
   name='Sonic Log Prediction Pipeline',
   description='An ML pipeline that performs Sonic Log Prediction prediction.'
)

# Define parameters to be fed into pipeline
def sonic_container_pipeline(
    data_path: str,
    holdout: bool,
    model_file: str
    ):
    
    # Define volume to share data between components.
    vop = dsl.VolumeOp(
    name="create_volume",
    resource_name="data-volume", 
    size="1Gi", 
    modes=dsl.VOLUME_MODE_RWO) # read write one
    
    # Create deploy component.
    train_tranform_container = train_tranform_op(data_path) \
                                    .add_pvolumes({data_path: vop.volume})

    # Create data transformation component.
    test_tranform_container = test_tranform_op(data_path) \
                                    .add_pvolumes({data_path: train_tranform_container.pvolume})
    
    # Create data transformation component.
    Outlier_removal_container = Outlier_removal_op(data_path) \
                                    .add_pvolumes({data_path: test_tranform_container.pvolume})
    
    # Create data transformation component.
    valid_tranform_container = valid_tranform_op(data_path) \
                                    .add_pvolumes({data_path: Outlier_removal_container.pvolume})
    
    # Create model training component.
    train_container = train_op(data_path, model_file, holdout) \
                                    .add_pvolumes({data_path: valid_tranform_container.pvolume})
    
    # Create model validation component.
    valid_container = valid_op(data_path, model_file, holdout) \
                                    .add_pvolumes({data_path: train_container.pvolume})
    
    # Create model training component.
    predict_container = predict_op(data_path, model_file) \
                                    .add_pvolumes({data_path: valid_container.pvolume})
    
    
    # Print the result of the prediction
    churn_result_container = dsl.ContainerOp(
        name="print_prediction",
        image='library/bash:4.4.23', # Image tag for the Docker container to be used.
        pvolumes={data_path: predict_container.pvolume}, # the name displayed for the component execution during runtime.
        arguments=['cat', f'{data_path}/result.txt']
    )

## Run the Pipeline

In [24]:
pipeline_func = sonic_container_pipeline

In [25]:
DATA_PATH = out_dir # mount your filesystems or directory
MODEL_PATH = 'model_file'

In [26]:
experiment_name = 'dt_prediction_kubeflow'
run_name = pipeline_func.__name__ + ' run'

arguments = {"data_path":DATA_PATH,
            "holdout":holdout,
            "model_file":MODEL_PATH}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func, '{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

/home/jovyan/.local/lib/python3.6/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


http://localhost:8888/pipeline#/experiments/details/644a33c5-3114-4b3c-b6de-3b0b6467eafa

http://localhost:8888/pipeline#/runs/details/2aadb518-ead4-4035-90ea-011a44bfe16d